In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt



**Training Model**

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")

In [4]:
train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [5]:
train.describe()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.00000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,10378.50000,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5992.46278,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,0.00000,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5189.25000,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,10378.50000,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,15567.75000,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,20757.00000,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


In [6]:
train.isnull().values.any()

False

In [7]:
def label_encoding(df,is_training_set=False):
    df['Gender'].replace(['Female', 'Male'],
                        [0, 1], inplace=True)
    df['family_history_with_overweight'].replace(['no', 'yes'],
                            [0, 1], inplace=True)
    df['FAVC'].replace(['no', 'yes'],
                            [0, 1], inplace=True)
    df['CAEC'].replace(['no', 'Sometimes', 'Frequently', 'Always'],
                            [0, 1, 2, 3], inplace=True)
    df['SMOKE'].replace(['no', 'yes'],
                            [0, 1], inplace=True)
    df['SCC'].replace(['no', 'yes'],
                            [0, 1], inplace=True)
    df['CALC'].replace(['no', 'Sometimes','Frequently','Always'],
                            [0, 1,2,3], inplace=True)
    df['MTRANS'].replace(['Public_Transportation', 'Automobile','Walking','Motorbike','Bike'],
                            [0, 1,2,3,4], inplace=True)
    
    if is_training_set:
        df['NObeyesdad'].replace(['Insufficient_Weight', 'Normal_Weight','Obesity_Type_I','Obesity_Type_II','Obesity_Type_III','Overweight_Level_I','Overweight_Level_II'],
                        [0,1,2,3,4,5,6], inplace=True)
    
    return df

In [8]:
train.drop(['id'],axis=1,inplace=True)

In [9]:
train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [10]:
train = label_encoding(train,True)

/tmp/ipykernel_18/1675263398.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Gender'].replace(['Female', 'Male'],
/tmp/ipykernel_18/1675263398.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'].replace(['Female', 'Male'],
/tmp/ipykernel_18/1675263398.py:4: FutureWarning: A valu

In [11]:
train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1,0,2.763573,0,0.000000,0.976473,1,0,6
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2,0,2.000000,0,1.000000,1.000000,0,1,1
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1,0,1.910378,0,0.866045,1.673584,0,0,0
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1,0,1.674061,0,1.467863,0.780199,1,0,4
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1,0,1.979848,0,1.967973,0.931721,1,0,6


In [12]:
scaler = StandardScaler()

In [13]:
train[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']] = scaler.fit_transform(train[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']])

In [14]:
train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,0.105699,-0.002828,-0.235713,1,1,-0.836279,0.314684,1,0,1.206594,0,-1.171141,0.597438,1,0,6
1,0,-1.027052,-1.606291,-1.170931,1,1,-0.836279,0.338364,2,0,-0.048349,0,0.021775,0.636513,0,1,1
2,0,-1.027052,0.128451,-1.430012,1,1,-1.060332,-1.913423,1,0,-0.195644,0,-0.138022,1.755239,0,0,0
3,0,-0.507929,0.120090,1.644770,1,1,1.039171,0.338364,1,0,-0.584035,0,0.579896,0.271455,1,0,4
4,1,1.371197,2.450367,0.224054,1,1,0.438397,-1.119801,1,0,-0.081469,0,1.176486,0.523111,1,0,6


In [15]:
X = train.drop(['NObeyesdad'],axis=1)
y = train['NObeyesdad']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
model_svc = SVC()
model_svc.fit(X_train,y_train)
y_pred_svc = model_svc.predict(X_test)

In [18]:
print(accuracy_score(y_test,y_pred_svc))

0.8749197174052665


In [19]:
print(classification_report(y_test,y_pred_svc))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       755
           1       0.85      0.82      0.84       907
           2       0.86      0.85      0.86       858
           3       0.96      0.98      0.97      1005
           4       1.00      1.00      1.00      1207
           5       0.69      0.74      0.72       733
           6       0.75      0.73      0.74       763

    accuracy                           0.87      6228
   macro avg       0.86      0.86      0.86      6228
weighted avg       0.88      0.87      0.87      6228



In [20]:
print(confusion_matrix(y_test,y_pred_svc))

[[ 692   59    1    0    0    2    1]
 [  58  745    0    0    0   96    8]
 [   2    0  729   32    3   29   63]
 [   0    0   16  983    1    0    5]
 [   1    0    1    1 1204    0    0]
 [   2   60   24    0    0  540  107]
 [   0   13   76    8    0  110  556]]


**Testing Model**

In [21]:
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")

In [22]:
test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [23]:
ids = test['id']

In [24]:
ids

0        20758
1        20759
2        20760
3        20761
4        20762
         ...  
13835    34593
13836    34594
13837    34595
13838    34596
13839    34597
Name: id, Length: 13840, dtype: int64

In [25]:
test.drop(['id'],axis=1,inplace=True)

In [26]:
test = label_encoding(test)

/tmp/ipykernel_18/1675263398.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Gender'].replace(['Female', 'Male'],
/tmp/ipykernel_18/1675263398.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'].replace(['Female', 'Male'],
/tmp/ipykernel_18/1675263398.py:4: FutureWarning: A valu

In [27]:
test.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,0,2.825629,0,0.855400,0.000000,1,0
1,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,0,3.000000,0,1.000000,0.000000,1,0
2,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,0,2.621877,0,0.000000,0.250502,1,0
3,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,0,2.786417,0,0.094851,0.000000,1,0
4,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,0,2.653531,0,0.000000,0.741069,1,0


In [28]:
test[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']] = scaler.fit_transform(test[['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']])

In [29]:
test.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,0.508163,1.682789,1.273786,1,1,0.932524,0.350808,1,0,1.298389,0,-0.141769,-1.005016,1,0
1,0,-0.509128,-1.114658,-0.818988,1,1,-0.833161,-2.462523,1,0,1.583678,0,0.030307,-1.005016,1,0
2,0,0.353000,-0.626191,0.927432,1,1,1.047997,0.350808,1,0,0.965029,0,-1.159701,-0.592995,1,0
3,1,-0.512705,-1.642760,0.623672,1,1,-0.833161,0.319733,1,0,1.234234,0,-1.046828,-1.005016,1,0
4,0,0.353000,-0.805996,0.668336,1,1,1.047997,0.350808,1,0,1.016818,0,-1.159701,0.213881,1,0


In [30]:
classification = {
     0: "Insufficient_Weight",
     1: "Normal_Weight",
     2: "Obesity_Type_I",
     3: "Obesity_Type_II",
     4: "Obesity_Type_III",
     5: "Overweight_Level_I",
     6: "Overweight_Level_II"
}

In [31]:
predictions = model_svc.predict(test)

In [32]:
NObeyesdad_predicted = []
for pre in predictions:
    NObeyesdad_predicted.append(classification[pre])

In [33]:
# Create DataFrame submission
submission = pd.DataFrame({'id': ids, 'NObeyesdad': NObeyesdad_predicted})

# Display the first few rows of the submission DataFrame
print(submission.head())

      id          NObeyesdad
0  20758     Obesity_Type_II
1  20759  Overweight_Level_I
2  20760    Obesity_Type_III
3  20761      Obesity_Type_I
4  20762    Obesity_Type_III


In [34]:
submission.to_csv('submission.csv', index=False)